In [1]:
import numpy as np
import pandas as pd

import requests
from pprint import pprint

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import datetime
from datetime import datetime

from functools import reduce

import json

from urllib.request import urlopen

import folium

import warnings
warnings.filterwarnings("ignore")

## Extracción datos

Con esta función pretendemos automatizar la extracción de datos respecto a la demanada nacional de energía diaria desde 2011 hasta la actualidad.

In [2]:
def extraccion_demanda_nac():
    search_url = "https://apidatos.ree.es/es/datos/"
    Data_gen=[]
    for x in range(2011,2024):
        start_date=f"{x}-01-01T00:00"
        end_date=f"{x}-12-31T23:59"
        query =f"start_date={start_date}&end_date={end_date}&time_trunc=day"
        endpoint =f"{search_url}demanda/evolucion?{query}"
        print(f"Endpoint: {endpoint}")
        response = requests.get(url = endpoint)
        print(f"response: {response.status_code}")
        data = response.json()
        Data_gen.extend(data["included"][0]["attributes"]["values"])        
    DF = pd.DataFrame(Data_gen)
    DF.drop("percentage", axis="columns", inplace=True)
    DF.rename(columns={"value":"Demanda energética","datetime":"Fecha"},inplace=True)
    DF["Energia Consumida Mw/h"]=DF["Demanda energética"].apply(lambda x : round(x/1000,2))
    DF.drop("Demanda energética", axis="columns", inplace=True)
    DF["Fecha"]=DF["Fecha"].apply(lambda x : x.split("T")[0])
    DF["Fecha"] = pd.to_datetime(DF["Fecha"])
    return DF

In [3]:
%%time
demanda_nacional = extraccion_demanda_nac()
demanda_nacional

Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2012-01-01T00:00&end_date=2012-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2013-01-01T00:00&end_date=2013-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2014-01-01T00:00&end_date=2014-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2015-01-01T00:00&end_date=2015-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2016-01-01T00:00&end_date=2016-12-31T23:59&time_trunc=day
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2017-01-01T00:00&end_date=2017-12-31T23:59&time_trunc=day
respon

,Fecha,Energia Consumida Mw/h
0,2011-01-01,605.99
1,2011-01-02,641.86
2,2011-01-03,801.30
3,2011-01-04,833.25
4,2011-01-05,803.48
...,...,...
4608,2023-08-14,677.55
4609,2023-08-15,630.29
4610,2023-08-16,698.40
4611,2023-08-17,705.51


In [4]:
demanda_nacional.to_csv('demanda_nacional.csv', index=False)

In [5]:
demanda_nacional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4613 entries, 0 to 4612
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Fecha                   4613 non-null   datetime64[ns]
 1   Energia Consumida Mw/h  4613 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 72.2 KB


Con esta función pretendemos automatizar la extracción de datos respecto a la demanada de energía anual desde 2011 hasta la actualidad para la comunidad que se desee.

In [6]:
def extraccion_demanda_com(comunidad):
    search_url = "https://apidatos.ree.es/es/datos/"
    Data_gen=[]
    dict_comunidades={"Andalucia": 4, "Aragon": 5, "Cantabria": 6, "Castilla-La Mancha": 7,
                         "Castilla-Leon": 8, "Cataluña": 9, "Pais Vasco": 10, "Asturias": 11,
                         "Ceuta": 8744, "Melilla": 8745, "Madrid": 13, "Navarra": 14, "Valencia": 15,
                         "Extremadura": 16, "Galicia": 17, "Baleares": 8743, "Canarias": 8742, 
                         "La Rioja": 20, "Murcia": 21}
    
    if comunidad in dict_comunidades.keys():
        for x in range(2011,2024):
            start_date=f"{x}-01-01T00:00"
            end_date=f"{x}-12-31T23:59"
            query =f"start_date={start_date}&end_date={end_date}&time_trunc=month&geo_limit=ccaa&geo_ids={dict_comunidades.get(comunidad)}"
            endpoint =f"{search_url}demanda/evolucion?{query}"
            print(f"Endpoint: {endpoint}")
            response = requests.get(url = endpoint)
            print(f"response: {response.status_code}")
            data = response.json()
            Data_gen.extend(data["included"][0]["attributes"]["values"])
        df_demanda_comunidad = pd.DataFrame(Data_gen)
        df_demanda_comunidad.drop("percentage", axis="columns", inplace=True)
        df_demanda_comunidad.rename(columns={"value":"Demanda energética","datetime":"Fecha"},inplace=True)
        df_demanda_comunidad[comunidad]=df_demanda_comunidad["Demanda energética"].apply(lambda x : round(x/1000,2))
        df_demanda_comunidad.drop("Demanda energética", axis="columns", inplace=True)
        df_demanda_comunidad["Fecha"]=df_demanda_comunidad["Fecha"].apply(lambda x : x.split("T")[0])
        df_demanda_comunidad["Fecha"] = pd.to_datetime(df_demanda_comunidad["Fecha"])
    return df_demanda_comunidad

Esta función extrae de manera automática los datos de la demanada de energía para todas las comunidades y devuelve un dataframe con dichos datos.

In [7]:
def get_all_data():
    dict_comunidades={"Andalucia": 4, "Aragon": 5, "Cantabria": 6, "Castilla-La Mancha": 7,
                         "Castilla-Leon": 8, "Cataluña": 9, "Pais Vasco": 10, "Asturias": 11,
                         "Ceuta": 8744, "Melilla": 8745, "Madrid": 13, "Navarra": 14, "Valencia": 15,
                         "Extremadura": 16, "Galicia": 17, "Baleares": 8743, "Canarias": 8742, 
                         "La Rioja": 20, "Murcia": 21}
    
    dict_comunidades = list(dict_comunidades.keys())
    
    df = pd.DataFrame()
    
    for comunidad in dict_comunidades:
        
        df_ = extraccion_demanda_com(comunidad)
        
        df = pd.concat([df, df_], axis = 1)
    columna_fecha = df.iloc[:, 0]  
    identical_columns = df.columns[df.T.duplicated()]
    df_aux=df.drop(columns=identical_columns[1:])
    df_aux.insert(0, "Fecha", columna_fecha, allow_duplicates=True)
    return df_aux

In [ ]:
%%time
df_general = get_all_data()

Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=4
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2012-01-01T00:00&end_date=2012-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=4
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2013-01-01T00:00&end_date=2013-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=4
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2014-01-01T00:00&end_date=2014-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=4
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2015-01-01T00:00&end_date=2015-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=4
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2016-01-01T00:00&end_date=2016-12-31T23:59&time_trunc=month&geo_limit=cca

response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2020-01-01T00:00&end_date=2020-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=7
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2021-01-01T00:00&end_date=2021-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=7
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=7
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2023-01-01T00:00&end_date=2023-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=7
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2012-01-01T00:00&end_date=2012-12-31T23:59&time_trunc=month

response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2016-01-01T00:00&end_date=2016-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=11
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2017-01-01T00:00&end_date=2017-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=11
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2018-01-01T00:00&end_date=2018-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=11
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2019-01-01T00:00&end_date=2019-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=11
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2020-01-01T00:00&end_date=2020-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=11
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2021-01-01T00:00&end_date=2021-12-31T23:59&time_trunc=

response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2012-01-01T00:00&end_date=2012-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=14
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2013-01-01T00:00&end_date=2013-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=14
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2014-01-01T00:00&end_date=2014-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=14
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2015-01-01T00:00&end_date=2015-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=14
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2016-01-01T00:00&end_date=2016-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=14
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2017-01-01T00:00&end_date=2017-12-31T23:59&time_trunc=

response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2021-01-01T00:00&end_date=2021-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=17
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2022-01-01T00:00&end_date=2022-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=17
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2023-01-01T00:00&end_date=2023-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=17
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2011-01-01T00:00&end_date=2011-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8743
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2012-01-01T00:00&end_date=2012-12-31T23:59&time_trunc=month&geo_limit=ccaa&geo_ids=8743
response: 200
Endpoint: https://apidatos.ree.es/es/datos/demanda/evolucion?start_date=2013-01-01T00:00&end_date=2013-12-31T23:59&time_tr

In [ ]:
df_general

In [ ]:
df_general.to_csv('demanda_comunidades.csv', index=False)